TODO: 
- všechny soubory
- správně dělení/násobení
- optimalizace
- uložení do souborů (data jsou již ve správném json tvaru)

In [1]:
import itertools

# filenames = [
#     "BNBBTC-2020-02-05",
#     "BCHBNB-2020-02-05",
#     "BCHBTC-2020-02-05"
# ]

currencies = ["USDT", "BTC", "LTC", "ETH", "XRP", "BCH", "EOS", "BNB", "TRX", "XMR"]

def get_comb(files):
    '''Generator iterating over all possible combinations and yielding valid combinations'''
    all_combinations = itertools.combinations(currencies, 3)
    for comb in all_combinations:
        fit = [f for f in files if set(f).issubset(comb)]
        if len(fit) == 3:
            yield (fit, ''.join(comb))
    yield None

def filenames_combinations(files):
    '''Generator returning 3 filenames for a triangle'''
    print(files)
    date = files[0][-11::]
    for i in range(len(files)):
        f = files[i][0:-11:]
        if len(f) == 6:
            files[i] = (f[0:3], f[-3::])
        elif len(f) == 7:
            files[i] = (f[0:3], f[-4::])
            
    gen = get_comb(files)  # generator 
    comb, name = next(gen)
    while comb is not None:
        ret = []
        for c in comb:
            ret.append(c[0] + c[1] + date)  # converison back to file name
        yield (ret, name)
        comb, name = next(gen)
    yield None
    
def split_string_to_array(field):
    field = field[0:-1]
    tmp = field.split("'")[1::2]
    return [float(i) for i in tmp]

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

MYPATH = "./data/"


class Files_reader:
    def __init__(self):
        self.all_files = None
    
    
    def fetch_filenames(self):
        try:
            self.all_files = [f for f in listdir(MYPATH) if isfile(join(MYPATH, f))] 
        except e:
            print(e)

    
    def files_selection(self):
        try:
            date = self.all_files[0][-11::1]
        except: 
            return None
        match = []
        does_not_match = []
        for f in self.all_files:
            if f[-11::1] == date:
                match.append(f)
            else:
                does_not_match.append(f)
        self.all_files = does_not_match
        return match

    def files_combinations(files):
        pass
        
    
    def read_files(self):
        if self.all_files is None:
            self.fetch_filenames()
        selected = self.files_selection()
        if selected is None or self.all_files is None:
            return None
        generator = filenames_combinations(selected)
        filenames, name = next(generator)
        while filenames is not None:
            dataframes = []
            for filename in filenames:
                tmp = pd.read_csv(MYPATH + filename, sep=';', usecols=[0, 1, 2, 3], header=None, names=['id', 'demand', 'supply', 'timestamp'])
                tmp["currency"] = filename
                dataframes.append(tmp)
            yield dataframes, name
            filenames, name = next(a)
        yield None

In [3]:
class Currency_pair:
    def __init__(self, row):
        self._row = row
        self._trade_id = row["id"]
        self._supply = split_string_to_array(str(row['supply'].values))
        self._demand = split_string_to_array(str(row['demand'].values))
        self._timestamp = row["timestamp"]
        self._currency = row["currency"]
        
    @property
    def trade_id(self):
        return self._trade_id
    
    @property
    def supply(self):
        return self._supply
    
    @property
    def demand(self):
        return self._demand
    
    @property
    def timestamp(self):
        return self._timestamp
    
    @property
    def currency(self):
        return self._currency
    
    def printout(self):
        print(self.trade_id)
        print(self.supply)
        print(self.demand)
        print(self.timestamp)
        
    def to_json(self):
        return self._row.to_json()


In [4]:
import numpy as np
from copy import deepcopy

class Arbitrage:
    def __init__(self):
        self.files_reader = Files_reader()
        self.dataframes = None
        self.current = None
        self.stop = False
        self.arbitrages = []
        self.output_name = None
        
    def initialize(self):
        generator = self.files_reader.read_files()  # dataframes generator
        self.dataframes, self.output_name = next(generator)
#         while b is not None:
#             self.dataframes = next(a)
        self.current = [None]*len(self.dataframes)
        print(len(self.dataframes))
        for i in range(len(self.dataframes)):
            self.get_next(i)
        
    def get_next(self, index):
        if len(self.dataframes[index]) == 1:
            self.stop = True  # todo - make this more advanced
        self.current[index] = Currency_pair(self.dataframes[index].head(1))
        self.dataframes[index].drop(self.dataframes[index].index[:1], inplace=True)

    def get_oldest(self):
        return np.argmin([float(self.current[0].timestamp), float(self.current[1].timestamp), float(self.current[2].timestamp)])

    def run(self):
        output = open(self.output_name, "a")   
        output.write("[")
        coma = ''
        while not self.stop:
            index = self.get_oldest()
            self.get_next(index)
            if self.detection():
                out = []
                for pair in self.current:
                    out.append(pair.to_json())
                out = "%s{score: %s, pairs: [%s,%s,%s]}"%(coma, "1.1", out[0], out[1], out[2])
                coma = ','
                self.arbitrages.append(out)
                output.write(out)
        output.write("]")
        output.close()
            
    def detection(self):
        percentage = self.current[1].supply[0] / self.current[0].supply[0] / self.current[2].supply[0]
        if percentage > 1:
            print(percentage)
            return True
#         print(percentage)
        return False


In [5]:
arb = Arbitrage()
arb.initialize()
arb.run()

['BCHBNB-2020-02-05', 'BCHBTC-2020-02-05', 'BNBBTC-2020-02-05']
3
1.001230663268255
1.001230663268255
1.001230663268255
1.001230663268255
1.0001730108131188
1.0001730108131188
1.0001145693663118
1.0001145693663118
1.0001145693663118
1.000215331501153
1.000215331501153
1.000215331501153
1.000215331501153
1.000215331501153
1.0003161139417296
1.0003161139417296
1.0003161139417296
1.00041691669418
1.00041691669418
1.00041691669418
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0004673256892769
1.0004673256892769
1.0004673256892769
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0005177397646454
1.0004673256892769
1.0004673256892769

1.0000188154235015
1.0004400570065526
1.0003673875755215
1.0003673875755215
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0003452963370743
1.0002758075671032
1.0002515784273016
1.0002515784273016
1.0002515784273016
1.0001788910078968
1.0001788910078968
1.000163119447082
1.000163119447082
1.000163119447082
1.000163119447082
1.0001873581868597
1.0001873581868597
1.0001873581868597
1.0005267005437477
1.0005267005437477
1.0005267005437477
1.0005267005437477
1.0005267005437477
1.0005267005437477
1.0005267005437477
1.0006258165673785
1.000601575426418
1.000601575426418
1.000601575426418
1.0005530931444968
1.0005530931444968
1.000652221602727
1.000627978060108
1.000627978060108
1.001272776499425
1.001272776499425
1.0002447757021975
1.0001093136539259
1.0001093136539259
1.000085081503469
1.000085081503469
1.000085081503469
1.000060849353012
1.000060849353012
1.000011243160088
1.0000123850520983
1.000012385

1.0002629773871679
1.0002629773871679
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.000165652058872
1.0003588041461737
1.0003588041461737
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.000334660135261
1.0002380840916103
1.0002871225817849
1.0002629773871679
1.0003120195021604
1.0001671412315396
1.0001671412315396
1.0001188484746661
1.0001678859273984
1.0001678859273984
1.0001195908026523
1.000168630696288
1.000168630696288
1.000168630696288
1.000168630696288
1.0002410769355639
1.0002901251911875
1.0002901251911875
1.0002901251911875
1.00033917825735
1.00033917825735
1.00033917825735
1.0003882361347598
1.0003882361347598
1.0007505206239442
1.0007505206239442
1.0007505206239442
1.0007505206239442
1.00075

1.0001046116368943
1.0001046116368943
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0000563812833239
1.0001057985082724
1.0001057985082724
1.0001057985082724
1.0001552206173112
1.0001552206173112
1.0000828679367746
1.000231152847752
1.000231152847752
1.000231152847752
1.000231152847752
1.000231152847752
1.000231152847752
1.000231152847752
1.000231152847752
1.0003794817384364
1.0003794817384364
1.0003794817384364
1.0003794817384364
1.0003794817384364
1.0003794817384364
1.0004783921083928
1.0004783921083928
1.0004542650099264
1.0004542650099264
1.0004542650099264
1.0004542650099264
1.0000587503765959
1.0000587503765959
1.0000587503765959
1.0000587503765959
1.0000587503765959
1.0000358163467977
1.0000358163467977
1.0000358163467977
1.0001346683031411
1

1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.000423083871616
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004471336596792
1.0004711834477429
1.0004711834477429
1.0004711834477429
1.000543332811934
1.000543332811934
1.000543332811934
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0005673825999977
1.0009040796328887
1.0009040796328887
1.00

...

1.0005357629085527
1.0005357629085527
1.0005857497072592
1.0005187282450567
1.0005187282450567
1.0005187282450567
1.0004517067828542
1.0004517067828542
1.0004517067828542
1.0004517067828542
1.0004517067828542
1.0004517067828542
1.0003400043458504
1.0003400043458504
1.0003400043458504
1.000250642396247
1.000048102731424
1.0000660846455007
1.0000660846455007
1.0002661778997313
1.0001545260961529
1.0001545260961529
1.0002045563360655
1.000070567469647
1.000070567469647
1.000170629560544
1.0001214650109318
1.0001214650109318
1.0002215171783
1.0000875350140057
1.0000875350140057
1.0001875937968987
1.0001205960122919
1.0001205960122919
1.000170629560544
1.000148295848431
1.000148295848431
1.0000982634174231
1.0000535982276855
1.0000535982276855
1.0000535982276855
1.0000634436120326
1.0000634436120326
1.0001134893230674
1.000627318935381
1.0000080413432262
1.000013402909325
1.000013402909325
1.000013402909325
1.000013402909325
1.000013402909325
1.000013402909325
1.0000634436120326
1.000018765

1.0000015451918918
1.0002706642441976
1.0002706642441976
1.0002706642441976
1.0002706642441976
1.0002706642441976
1.0002706642441976
1.0001361047180446
1.0001361047180446
1.0001361047180446
1.0000688249549683
1.0000688249549683
1.0000688249549683
1.0000463983672763
1.0000463983672763
1.0001471130455952
1.0001246841993212
1.0001246841993212
1.0000743254538933
1.0000967531708196
1.0000967531708196
1.0000067286147802
1.0000964434785142
1.0000964434785142
1.000051897736967
1.000051897736967
1.000051897736967
1.000051897736967
1.000051897736967
1.000051897736967
1.000051897736967
1.0000743254538933
1.0000743254538933
1.0000743254538933
1.0000743254538933
1.0000743254538933
1.0000743254538933
1.0000743254538933
1.0002076985032797
1.0002076985032797
1.0002076985032797
1.0002580944169446
1.0002580944169446
1.0002132096022975
1.0003140121429475
1.0003140121429475
1.0002691228047613
1.0004203579306767
1.0004203579306767
1.0004428059932788
1.0004932289964539
1.0004932289964539
1.0004707798024541


1.0000951770925077
1.0000951770925077
1.0000951770925077
1.0001173035815252
1.0001173035815252
1.0001173035815252
1.0001173035815252
1.0001173035815252
1.0001173035815252
1.0000730506034905
1.0000730506034905
1.0000730506034905
1.00010670681604
1.00010670681604
1.00010670681604
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000181920062767
1.0000682129242442
1.0000682129242442
1.0000682129242442
1.0000682129242442
1.0000239533056066
1.0000239533056066
1.0000239533056066
1.0000239533056066
1.0000239533056066
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000018234962877
1.0000297151813258
1.0002731552607067
1.0002731552607067
1.0002731552607067
1.0002731552607067
1.0002731552607067
1.000228893428092
1.000228893428092
1.000228893428092
1.

1.0001509059048932
1.0004502477233912
1.0004502477233912
1.0004502477233912
1.0005001554480106
1.000367761620374
1.000367761620374
1.0004176677197316
1.0004176677197316
1.0003956626239894
1.0004455726051622
1.0004455726051622
1.0004235664115806
1.0004235664115806
1.0004235664115806
1.0004235664115806
1.0004235664115806
1.0004235664115806
1.0004235664115806
1.0002738546991432
1.0002738546991432
1.0002958575995347
1.0002958575995347
1.0002958575995347
1.0003178604999263
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0000024373134635
1.0003312185803144
1.0003312185803144
1.0003312185803144
1.000309209092557
1.000309209092557
1.000309209092557
1.0005293039701324
1.0005293039701324
1.0005293039701324
1.0005293039701324
1.0005792305821068
1.0005792305821068
1.0005792305821068
1.000579230582106

1.0002464159520825
1.0002464159520825
1.0002464159520825
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0002027685183141
1.0003991819702716
1.0004992368967076
1.0004992368967076
1.0005428886958743
1.0006930151671198
1.0006930151671198
1.0006930151671198
1.0006930151671198
1.0006930151671198
1.0007366735160212
1.0007366735160212
1.0007366735160212
1.0007366735160212
1.0007366735160212
1.0007366735160212
1.0007366735160212
1.000036437938791
1.000036437938791
1.000036437938791
1.0001864059124883
1.0001864059124883
1.0000991372268664
1.0001491321847362
1.0001491321847362
1.0001927687088281
1.0000927794287413
1.0000927794287413
1.0001364115904894
1.0001864059124883
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000550901807304
1.0000549705724